In [1]:
import os
import numpy as np
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import tensorflow as tf
def preprocess(file_path):
    byte_img = tf.io.read_file(file_path)
    img  = tf.io.decode_jpeg(byte_img)
    img = tf.image.resize(img, (150, 150))
    img=  img / 255.0
    return img

In [3]:
base_dir = "Images"
train_dir = os.path.join(base_dir, 'Train')
test_dir = os.path.join(base_dir, 'Test')

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

test_datagen = ImageDataGenerator(rescale=1./255)

In [4]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

Found 150 images belonging to 3 classes.
Found 150 images belonging to 3 classes.


In [5]:
img_input = layers.Input(shape=(150, 150, 3))

x = layers.Conv2D(16, 3, activation='relu')(img_input)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(32, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(2)(x)

x = layers.Flatten()(x)
x = layers.Dense(512, activation='relu')(x)
output = layers.Dense(1, activation='sigmoid')(x)

model = Model(img_input, output)
model.compile(loss='binary_crossentropy',
            optimizer=RMSprop(learning_rate=0.001),
            metrics=['acc'])

In [6]:
history = model.fit_generator(
    train_generator,
    steps_per_epoch=5,
    epochs=10,
    validation_data=test_generator,
    validation_steps=4,
    verbose=2)

C:\Users\91798\AppData\Local\Temp\ipykernel_16408\676592146.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/10


5/5 - 11s - loss: 0.9862 - acc: 0.3889 - val_loss: 0.2784 - val_acc: 0.3750 - 11s/epoch - 2s/step
Epoch 2/10
5/5 - 6s - loss: 0.3479 - acc: 0.3222 - val_loss: 0.2339 - val_acc: 0.2875 - 6s/epoch - 1s/step
Epoch 3/10
5/5 - 5s - loss: -4.7173e-03 - acc: 0.3333 - val_loss: -2.1593e+00 - val_acc: 0.3125 - 5s/epoch - 1s/step
Epoch 4/10
5/5 - 5s - loss: -7.8024e-01 - acc: 0.3600 - val_loss: -1.3301e+00 - val_acc: 0.3750 - 5s/epoch - 1s/step
Epoch 5/10
5/5 - 5s - loss: -1.0116e+01 - acc: 0.3111 - val_loss: -7.6614e+00 - val_acc: 0.6000 - 5s/epoch - 1s/step
Epoch 6/10
5/5 - 5s - loss: -1.6191e+01 - acc: 0.3889 - val_loss: -6.2217e+01 - val_acc: 0.3375 - 5s/epoch - 1s/step
Epoch 7/10
5/5 - 5s - loss: -5.8167e+01 - acc: 0.3778 - val_loss: -1.1212e+02 - val_acc: 0.3125 - 5s/epoch - 1s/step
Epoch 8/10
5/5 - 5s - loss: -1.2311e+02 - acc: 0.3222 - val_loss: -2.4423e+02 - val_acc: 0.4875 - 5s/epoch - 1s/step
Epoch 9/10
5/5 - 6s - loss: -2.5132e+02 - acc: 0.3700 - val_loss: -6.8900e+02 - 

In [ ]:
for file in os.listdir(os.path.join('Images', 'Train')):
    for image in os.listdir(os.path.join('Images', 'Train', file)):
        input_img = preprocess(os.path.join('Images/Input_Image', 'input_image.jpeg'))
        validation_img = preprocess(os.path.join('Images', 'Train', file, image))
results = model.evaluate(input_img, validation_img, batch_size=128)

In [3]:
def preprocess(file_path):
    byte_img = tf.io.read_file(str(file_path))
    # img  = tf.io.decode_jpeg(byte_img)
    img = tf.image.decode_jpeg(byte_img)
    img = tf.image.resize(img, (150, 150))
    img=  img / 255.0
    return img

def verify(model, detection_threshold, verification_threshold):
    # Build results array
    results = []
    for file in os.listdir(os.path.join('Images', 'Train')):
        for image in os.listdir(os.path.join('Images', 'Train', file)):
            # Images/Input_Image/input_image.jpeg
            input_img = preprocess(os.path.join('Images/Input_Image', 'input_image.jpeg'))
            validation_img = preprocess(os.path.join('Images', 'Train', file, image))

        # input_img = tf.image.resize(input_img, (100, 100))
        # input_img=  input_img / 255.0
        
        # validation_img = tf.image.resize(validation_img, (100, 100))
        # validation_img=  validation_img / 255.0
        
        # Make Predictions
        result_input_img = model.predict(list(np.expand_dims([input_img], axis=1)))
        result_validation_img = model.predict(list(np.expand_dim([input_img]), axis=1))
        # results.append(result)
        print(result_input_img, result_validation_img)

    # Detection Threshold: Metric above which a prediciton is considered positive
    detection = np.sum(np.array(result) > detection_threshold)
    print(detection) # 3 

    # Verification Threshold: Proportion of positive predictions / total positive samples
    verification = detection / len(os.listdir(os.path.join("Images", 'Train', 'Bharat')))
    print(verification) # 0.02
    verified = verification > verification_threshold

    return results, verified

In [4]:
import cv2
import time
import os
cap = cv2.VideoCapture(0)

pTime = 0
while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Unable to capture frame from webcam.")
        break
    # FPS
    cTime = time.time()
    fps = 1/ (cTime - pTime)
    pTime = cTime
    cv2.putText(frame, f"FPS: {int(fps)}", (20, 70), cv2.FONT_HERSHEY_PLAIN, 3, (0, 255, 0), 3)

    cv2.imshow('Face Detection', frame)
    if cv2.waitKey(1) == ord('q'):
        break
    if cv2.waitKey(1) == ord('v'):
        cv2.imwrite(os.path.join("Images", "Input_Image", "input_image.jpeg"), frame)
        results, verified = verify(model, 0.9, 0.01)
        print(verified)

cap.release()
cv2.destroyAllWindows()

NameError: name 'model' is not defined

In [57]:
input_img = preprocess(os.path.join('Images/Input_Image', 'input_image.jpeg'))
np.expand_dims([input_img], axis=1)

array([[[[[0.09789542, 0.09933333, 0.08717647],
          [0.10905883, 0.10905883, 0.11611765],
          [0.10424836, 0.10424836, 0.1120915 ],
          ...,
          [0.12222207, 0.1300652 , 0.1183005 ],
          [0.12662773, 0.12584312, 0.12623543],
          [0.1495684 , 0.13070562, 0.14614354]],

         [[0.09777778, 0.09777778, 0.09490196],
          [0.10941177, 0.10941177, 0.11647059],
          [0.10313725, 0.10313725, 0.11098039],
          ...,
          [0.12666667, 0.1345098 , 0.1227451 ],
          [0.12482372, 0.13188225, 0.12835298],
          [0.1340263 , 0.13115023, 0.1404314 ]],

         [[0.11176471, 0.10888889, 0.1103268 ],
          [0.10392157, 0.10392157, 0.11176471],
          [0.10915032, 0.10915032, 0.11699346],
          ...,
          [0.1232027 , 0.1232027 , 0.11535956],
          [0.12960792, 0.13745105, 0.13352948],
          [0.13333334, 0.13333334, 0.14117648]],

         ...,

         [[0.08254901, 0.08183007, 0.07398693],
          [0.08215687,